# Dependencies

In [37]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload


<class 'torch.device'>
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:

import datasets_handler
import zeroberto
import evaluation_metrics
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from zeroberto import ZeroBERTo
import pandas as pd

<class 'torch.device'>


# Dataset

In [39]:
which_dataset = 'ag_news'
# which_dataset = 'folhauol'
sample_size = 1000
# hypothesis = "this text is about {}."
hypothesis = "{}."


raw_data, data_col, class_col = datasets_handler.getDataset(which_dataset)
raw_data[data_col] = raw_data[data_col].dropna()
classes_list = raw_data[class_col].drop_duplicates().to_list()
raw_data = evaluation_metrics.Encoder(raw_data,[class_col])

Found cached dataset ag_news (/Users/alealcoforado/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


  0%|          | 0/2 [00:00<?, ?it/s]

In [40]:
train_dataset = raw_data.sample(sample_size,random_state=422)
X = train_dataset[data_col].to_list()
Y_class =  train_dataset[class_col].to_list()
Y_code =  train_dataset[class_col+"_code"].to_list()
pd.Series(zip(Y_class,Y_code)).drop_duplicates()

0    (science and technology, 1)
1                    (sports, 2)
2                  (business, 0)
3                     (world, 3)
dtype: object

In [41]:
classes_list_encoded = evaluation_metrics.Encoder(pd.DataFrame({"classes":classes_list}))['classes_code']

# Model

In [42]:
model = zeroberto.ZeroBERTo(classes_list=classes_list,hypothesis_template=hypothesis)

['business.', 'science and technology.', 'sports.', 'world.']


In [43]:
test = "O país está na merda. Mas tenho duas frases."
model.encode(test)

tensor([ 3.2136e-02,  1.9865e-02,  3.1469e-02,  2.7527e-02,  8.4466e-03,
         1.8062e-02,  3.1492e-02, -7.3412e-03,  1.8927e-02, -6.2870e-02,
        -2.3049e-02,  1.4962e-02, -5.6942e-02, -9.1903e-03,  5.8556e-02,
         5.2277e-02, -2.4588e-02, -3.5469e-02, -1.0119e-02, -8.1159e-03,
         9.9538e-03, -3.0489e-02,  2.0677e-02, -2.1111e-02,  3.8647e-02,
        -9.4266e-03,  3.3564e-02, -2.7355e-02, -6.3083e-02, -1.0667e-02,
         4.4673e-02, -4.4717e-02, -8.1970e-05, -1.5298e-02, -9.6788e-03,
         4.6747e-02, -4.3560e-02, -8.1932e-03,  4.4921e-02, -1.4903e-02,
         6.3074e-02, -1.8807e-03, -1.4162e-02, -1.0791e-02, -3.3622e-02,
         1.5255e-02,  9.0523e-03,  3.4798e-02, -2.9422e-02, -4.9509e-02,
        -3.8473e-02,  5.7271e-02, -3.5727e-03,  5.0541e-02, -7.9862e-03,
         5.1902e-02,  2.0824e-02, -7.0982e-02, -3.9434e-02, -1.1077e-02,
         6.8836e-02, -4.2466e-02,  4.8153e-02,  9.3536e-03, -6.1508e-02,
         2.0166e-03, -5.3895e-02, -1.8302e-02, -3.4

In [44]:
X_emb = model.encode(X)

In [45]:
classes_list_emb = model.encode(classes_list)


In [46]:
model.hypothesis_template,model.classes,

('{}.', ['business', 'science and technology', 'sports', 'world'])

# KMeans

In [47]:
from sklearn.cluster import KMeans
import numpy as np


In [48]:
cluster_centers = np.array(classes_list_emb,dtype=np.double)
print(cluster_centers)


[[ 0.02597696  0.03260607  0.0491689  ... -0.0497164  -0.04720933
  -0.00918514]
 [-0.00667754 -0.00183611  0.07915001 ... -0.03521105 -0.02245408
  -0.00815121]
 [ 0.0729172  -0.00049009  0.08419071 ...  0.02738611 -0.02243059
   0.01396182]
 [ 0.00950092 -0.02665639  0.10919806 ... -0.02656817 -0.00534643
  -0.0236638 ]]


In [49]:
# kmeans = KMeans(n_clusters=2, init=cluster_centers, n_init=1)
km1 = KMeans(n_clusters=len(classes_list), n_init=1, init=cluster_centers,max_iter = 600, random_state=422)

km1.fit(cluster_centers)

KMeans(init=array([[ 0.02597696,  0.03260607,  0.0491689 , ..., -0.0497164 ,
        -0.04720933, -0.00918514],
       [-0.00667754, -0.00183611,  0.07915001, ..., -0.03521105,
        -0.02245408, -0.00815121],
       [ 0.0729172 , -0.00049009,  0.08419071, ...,  0.02738611,
        -0.02243059,  0.01396182],
       [ 0.00950092, -0.02665639,  0.10919806, ..., -0.02656817,
        -0.00534643, -0.0236638 ]]),
       max_iter=600, n_clusters=4, n_init=1, random_state=422)

In [36]:
cluster_centers.shape

(4, 768)

In [37]:
# km1.score(X_emb,Y_code)

In [38]:

y_pred = km1.predict(X_emb)

In [39]:
try: print(all_metrics)
except: pass
all_metrics = evaluation_metrics.get_metrics(y_pred,Y_code)
print(all_metrics)

{'weighted': [{'accuracy': 0.304}, {'precision': 0.49178037098560357}, {'recall': 0.304}, {'f1': 0.2557830402594587}], 'macro': [{'accuracy': 0.304}, {'precision': 0.4900401439645626}, {'recall': 0.3039353107691872}, {'f1': 0.2533959236792093}]}
{'weighted': [{'accuracy': 0.462}, {'precision': 0.5651428393296906}, {'recall': 0.462}, {'f1': 0.43117904518349887}], 'macro': [{'accuracy': 0.462}, {'precision': 0.562462221242498}, {'recall': 0.46259378236729753}, {'f1': 0.4292006626151025}]}


In [40]:
def getTopLabel(row):  return row.idxmax()
def getTopProb(row):   return row.max()

y_probs = km1.transform(X_emb)
y_probs

array([[1.36662048, 1.30808537, 1.40733463, 1.36974043],
       [1.47019239, 1.46231025, 1.4774647 , 1.52124674],
       [1.30253007, 1.32461061, 1.43250614, 1.35873981],
       ...,
       [1.33169989, 1.35944845, 1.4107733 , 1.45561049],
       [1.44974579, 1.4281211 , 1.45022305, 1.51886671],
       [1.42201143, 1.48008053, 1.48927879, 1.5010865 ]])

In [41]:
df_probs = pd.DataFrame(y_probs,columns=classes_list)

label_results = df_probs.apply(getTopLabel,axis=1)
prob_results = df_probs.apply(getTopProb,axis=1)

In [42]:
label_results_df = pd.Series(label_results,name='prediction')
true_labels_df = raw_data[raw_data.index.isin(train_dataset.index)][class_col]
label_results_df.index=true_labels_df.index

final_result_df = pd.concat([true_labels_df,label_results_df],axis=1)
vector_true = ( final_result_df[class_col].astype(str).str.strip().str.lower() 
               == final_result_df['prediction'].astype(str).str.strip().str.lower() )
final_result_df_encoded = evaluation_metrics.Encoder(final_result_df,['prediction',class_col])
df_predictions_probs = pd.concat([final_result_df_encoded,
                                  pd.Series(prob_results,name='top_probability'),
                                  pd.Series(vector_true,name='right_wrong')],axis=1)

In [43]:
true_labels_df

59                      business
70                      business
445       science and technology
840                        world
851                        world
                   ...          
126596                  business
126759    science and technology
127454                    sports
127531                    sports
127563                    sports
Name: label_text, Length: 1000, dtype: object

In [44]:
label_results_df

59        sports
70         world
445       sports
840        world
851        world
           ...  
126596     world
126759    sports
127454     world
127531     world
127563     world
Name: prediction, Length: 1000, dtype: object

In [45]:
def get_top_n_results(dataframe_results,top_n=1):
    df_top_n = dataframe_results.sort_values(['top_probability','prediction'], ascending=True).groupby('prediction').head(top_n)
    # print(df_top_n)
    accuracy_top_n = df_top_n.right_wrong.sum()/len(df_top_n.right_wrong)
    # evaluation_metrics.get_metrics(df_top_n['prediction_code'],Y_code,metrics=["accuracy"])

    accuracy = { "accuracy in top {}".format(top_n) : round(accuracy_top_n,3)}
    print(accuracy)
    return 

get_top_n_results(df_predictions_probs,top_n = 1)
get_top_n_results(df_predictions_probs,top_n = 2)
get_top_n_results(df_predictions_probs,top_n = 3)
get_top_n_results(df_predictions_probs,top_n = 4)
get_top_n_results(df_predictions_probs,top_n = 5)
get_top_n_results(df_predictions_probs,top_n = 6)
get_top_n_results(df_predictions_probs,top_n = 7)
get_top_n_results(df_predictions_probs,top_n = 8)
get_top_n_results(df_predictions_probs,top_n = 9)
get_top_n_results(df_predictions_probs,top_n = 10)
get_top_n_results(df_predictions_probs,top_n = 11)
get_top_n_results(df_predictions_probs,top_n = 12)
get_top_n_results(df_predictions_probs,top_n = 13)
get_top_n_results(df_predictions_probs,top_n = 14)
get_top_n_results(df_predictions_probs,top_n = 15)
get_top_n_results(df_predictions_probs,top_n = 16)
get_top_n_results(df_predictions_probs,top_n = len(train_dataset))

{'accuracy in top 1': 0.25}
{'accuracy in top 2': 0.25}
{'accuracy in top 3': 0.333}
{'accuracy in top 4': 0.312}
{'accuracy in top 5': 0.3}
{'accuracy in top 6': 0.333}
{'accuracy in top 7': 0.286}
{'accuracy in top 8': 0.25}
{'accuracy in top 9': 0.25}
{'accuracy in top 10': 0.225}
{'accuracy in top 11': 0.205}
{'accuracy in top 12': 0.188}
{'accuracy in top 13': 0.212}
{'accuracy in top 14': 0.214}
{'accuracy in top 15': 0.217}
{'accuracy in top 16': 0.203}
{'accuracy in top 1000': 0.249}
